### Model architecture demonstration

In [88]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Dense, Flatten, Reshape, RandomFlip, RandomRotation, RandomZoom

In [89]:
train_dir = 'data/zooplankton/train'

In [90]:
CLASSES = ["Lucicutiidae", "Mecynocera", "Mysida", "Ostracoda", 
        "Pleuromamma", "Pontellidae", "Rhincalanidae", "Sapphirina", 
        "Scolecitrichidae", "Sergestidae", "Subeucalanidae", "Temoridae", 
        "Acartiidae", "Aetideidae", "Calocalanus", "Calyptopsis", 
        "Candaciidae", "Centropagidae", "Cladocera", "Copilia", 
        "Eucalanidae", "Euchaetidae", "Haloptilus", "Harpacticoida"]

#### Constuct training and validation sets

In [91]:
batch_size = 32
img_height = 90
img_width = 90
img_channels = 3

train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  seed=123,
  subset="training",
  validation_split=0.2,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  seed=123,
  subset="validation",
  validation_split=0.2,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 20142 files belonging to 24 classes.
Using 16114 files for training.
Found 20142 files belonging to 24 classes.
Using 4028 files for validation.


##### Generate "additional" images by augmenting existing ones

In [ ]:
augmentation = Sequential([
    RandomRotation(factor=0.2, input_shape=(img_height, img_width, img_channels)),
    RandomZoom(height_factor=0.2, width_factor=0.2),
    RandomFlip(mode='horizontal')
])

In [ ]:
model = Sequential([
    augmentation,

    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(CLASSES), activation='softmax')   # multiclass classification => softmax
])


In [94]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),     # SCCrossentrtopy loss for non-enocded, multiple classes
              metrics=['accuracy'])

model.build()

In [95]:
model.summary()

Model: "sequential_71"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_70 (Sequential)  (None, 90, 90, 3)         0         
                                                                 
 conv2d_60 (Conv2D)          (None, 88, 88, 32)        896       
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 44, 44, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_61 (Conv2D)          (None, 42, 42, 64)        18496     
                                                                 
 batch_normalization_45 (Bat  (None, 42, 42, 64)       256       
 chNormalization)                                                
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 21, 21, 64)     

##### Callback (stops training if no progress is being made and loads best epoch weights)

In [72]:
class zooplankton_callback(keras.callbacks.Callback):
    def __init__ (self, model, epochs):
        super(zooplankton_callback, self).__init__()
        self.model=model               
        self.epochs=epochs
        self.lowest_vloss=np.inf
        self.best_weights=self.model.get_weights()
        self.best_epoch=1
                
    def on_train_end(self, logs=None):  
        self.model.set_weights(self.best_weights) 
        
    def on_epoch_end(self, epoch, logs=None):  
        v_loss=logs.get('val_loss')  
        if v_loss< self.lowest_vloss:
            self.lowest_vloss=v_loss
            self.best_weights=self.model.get_weights() 
            self.best_epoch=epoch + 1

In [ ]:
epochs = 10

callbacks=[zooplankton_callback(model, epochs)]

model.fit(x=train_ds,  
            epochs=epochs,
            verbose=1,
            callbacks=callbacks,  
            validation_data=val_ds,
            validation_steps=None,
            shuffle=False,  
            initial_epoch=0)

#### Testing

In [85]:
test_dir = '/home/ttsonev/Desktop/Projects/zooplankton-files/zooplankton/test'

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  seed=123,
  subset="training",
  validation_split=0.2,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 2507 files belonging to 24 classes.
Using 2006 files for training.


In [ ]:
model.evaluate(test_ds)

#### Export model

In [ ]:
io_option = tf.saved_model.SaveOptions(experimental_io_device="/job:localhost")
model.save("model", options=io_option)